In [ ]:
!sudo apt-get update -y --quiet

In [ ]:
!sudo apt-get install -y --quiet tree libcurl4-openssl-dev zlib1g-dev samtools

In [3]:
!pip install biopython
!pip install pyBigWig
!pip install pysam

     |████████████████████████████████| 2.3 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 10.3 MB/s eta 0:00:01   |██████████▊                     | 4.9 MB 7.7 MB/s eta 0:00:02
     |████████████████████████████████| 64 kB 2.2 MB/s eta 0:00:011
  Created wheel for pyBigWig: filename=pyBigWig-0.3.17-cp38-cp38-linux_x86_64.whl size=223615 sha256=4c1366435ecdb96b8f095b219bb382368d9a912e27291106834628ebeb81a902
  Stored in directory: /home/jovyan/.cache/pip/wheels/6c/0f/66/71631d173ebddcb890b2b36cbab9325ed4913fd1d583c273ed
Successfully built pyBigWig
     |████████████████████████████████| 10.2 MB 4.6 MB/s eta 0:00:01    |████████████████▍               | 5.2 MB 2.8 MB/s eta 0:00:02


In [3]:
import os
import math
import numpy as np
import pysam

from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.SeqIO.QualityIO import FastqGeneralIterator

In [6]:
def bold_char(string, pos):
    string_list = list(string)
    BOLD = '\033[1m'
    END = '\033[0m'
    string_list.insert(pos, BOLD)
    string_list.insert(pos+2, END)
    return("".join(string_list))

def sam_flag_decoder(flag, logging=True):
    description = []
    for bit in range(12):
        bit_description = ""
        flag_component = flag & (1<<bit) # check if a particular bit is set on the binary flag
        binary_string = '{0:012b}'.format(flag) # convert to binary and set length to 12 bits
        if(flag_component): binary_string = bold_char(binary_string, 11-bit) # bold the bit if is set
        if(flag_component and bit == 0):  
            bit_description = "read paired - template having multiple segments in sequencing"
            description.append("read paired - template having multiple segments in sequencing")
        if(flag_component and bit == 1):  
            bit_description = "read mapped in proper pair - each segment properly aligned according to the aligner"
            description.append("read mapped in proper pair - each segment properly aligned according to the aligner")
        if(flag_component and bit == 2):  
            bit_description = "read unmapped - segment unmapped"
            description.append("read unmapped - segment unmapped")
        if(flag_component and bit == 3):  
            bit_description = "mate unmapped - next segment in the template unmapped"
            description.append("mate unmapped - next segment in the template unmapped")
        if(flag_component and bit == 4):  
            bit_description = "read reverse strand - SEQ being reverse complemented"
            description.append("read reverse strand - SEQ being reverse complemented")
        if(flag_component and bit == 5):  
            bit_description = "mate reverse strand - SEQ of the next segment in the template being reversed"
            description.append("mate reverse strand - SEQ of the next segment in the template being reversed")
        if(flag_component and bit == 6):  
            bit_description = "first in pair - the first segment in the template"
            description.append("first in pair - the first segment in the template")
        if(flag_component and bit == 7):  
            bit_description = "second in pair - the last segment in the template"
            description.append("second in pair - the last segment in the template")
        if(flag_component and bit == 8):  
            bit_description = "not primary alignment - secondary alignment"
            description.append("not primary alignment - secondary alignment")  
        if(flag_component and bit == 9):  
            bit_description = "read fails platform/vendor quality checks - not passing quality controls"
            description.append("read fails platform/vendor quality checks - not passing quality controls")
        if(flag_component and bit == 10): 
            bit_description = "read is PCR or optical duplicate"
            description.append("read is PCR or optical duplicate")
        if(flag_component and bit == 11): 
            bit_description = "supplementary alignment"
            description.append("supplementary alignment")
        if(logging):
            if(bit_description): print(binary_string + "  " + "{:<4}".format(str(flag_component)) + "  " + description[-1])
            else: print(binary_string + "  " + "{:<4}".format(str(flag_component)))
    return(" + ".join(description))

In [7]:
def sam_flag_to_bin(flag):
    print("{:<4}".format(str(flag)) + ": " + '{0:012b}'.format(flag))
    print("{:<4}".format(str(4095-flag)) + ": " + '{0:012b}'.format(4095-flag))

## PairedReads
For paired reads, 0'th bit HAS to be set. Hence all flags for paired reads HAVE to be odd. In other words, all even-numbered flags other than the above three (0, 4 and 16) are meaningless. 

For paired reads all flags in the intervals [65-127] and [193-255] relate to the first read of a pair. All other (odd) flags refer to the second read in a pair.

### Both reads have aligned
- 65 - 0001000001 - this is first read in pair and both reads aligned the forward strand.
- 129 - 0010000001 - this is second read of pair and both reads aligned the forward strand.
- 67 (0001000011) and 131 (0010000011) also mean the same as 65 and 129 with the added assurance that "the pair is properly aligned" meaning that they mapped within a proper distance from each other.
- 113 - 0001110001 - "this is the first read of a pair, both reads in pair were flipped and both mapped".
- 177 - 0001110001 - "this is the second read of a pair, both reads in pair were flipped and both mapped".
- 81 - 0001010001 - "this is the first read of pair, both reads mapped, we had to flip this read, but mate is in forward orientation".
- 161 - 0010100001 - "this is second read, this one is forward but we flipped its mate and both reads mapped".
- 163 (0010100011) and 83 (0001010011) are the same as 161 and 81 except "it is in a proper pair".
- 97 - 0001100001 - "this is first read, its mate is flipped but this is forward. Both mapped".
- 145 - 0010010001 - "this is second read. it is flipped but its mate is not. Both mapped".
- 99 (0001100011) and 147 (0010010011) are the same as 97 and 145 except with "proper mapping in pair".

### No reads have aligned
- 77 - 0001001101 - First in pair, both reads in pair unmapped. "All bad"
- 141 - 0010001101 - Second in pair and "all bad".

### Only one read has aligned
- 69 - 0001000101 - 1st read in pair. This read is unmapped but its mate is mapped.
- 137 - 0010001001 - 2nd in pair. Read is mapped but mate is unmapped.
- 73 - 0001001001 - 1st read in pair. This read is mapped but its mate is not.
- 133 - 0010000101 - 2nd in pair. Read unmapped but mate is mapped.

In [50]:
!ls notebook_test_data/include_and_exclude

reads-to-donor.bam	  reads-to-recipient.bam
reads-to-donor_MM.bam	  reads-to-recipient_MM.bam
reads-to-donor_MU.bam	  reads-to-recipient_MU.bam
reads-to-donor_MU_R1.bam  reads-to-recipient_MU_R1.bam
reads-to-donor_MU_R2.bam  reads-to-recipient_MU_R2.bam
reads-to-donor_UM.bam	  reads-to-recipient_UM.bam
reads-to-donor_UM_R1.bam  reads-to-recipient_UM_R1.bam
reads-to-donor_UM_R2.bam  reads-to-recipient_UM_R2.bam
reads-to-donor_UU.bam	  reads-to-recipient_UU.bam
reads-to-donor_UU_R1.bam  reads-to-recipient_UU_R1.bam
reads-to-donor_UU_R2.bam  reads-to-recipient_UU_R2.bam


In [53]:
!samtools view -@ 24 -f 73 notebook_test_data/include_and_exclude/reads-to-recipient.bam | cut -f2 | sort -n | uniq

73
77
121
329
377
2121
2169


In [54]:
!samtools view -@ 24 -f 133 notebook_test_data/include_and_exclude/reads-to-recipient.bam | cut -f2 | sort -n | uniq

133
141
181


In [55]:
!samtools view -@ 24 -f 73 notebook_test_data/include_and_exclude/reads-to-recipient.bam | head -n 5

SRR12091997.6	77	*	0	0	*	*	0	0	CTCTTNTCTGATGAACAGTTTAGGTGAAACTGATCTGGCACGTAACTGATAGACGTGTTTTACGCCGTCAGGACAA	AAAAA#EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEAAEEEEAEE	AS:i:0	XS:i:0
SRR12091997.33	77	*	0	0	*	*	0	0	CGGTGNGAATGTTTTGTATGCGTCAATATGCTTATTCAGCAAAATGACTTGATCTTTGAAATTTGGATCTTTGTCA	66AAA#EEEEAE6EEEEEEEEEEEEEE6EE/EEEEEEEEEEEEEEEE6EEEEEEEEEEEEEEEEEEEEEEEEEEE<	AS:i:0	XS:i:0
SRR12091997.34	77	*	0	0	*	*	0	0	CTCCANTGGTTGCTCTTCATCTAATTGAGAATATTTATTCTCAGTTAGTGACTTAGATAAATTTTTAATTATGAGG	AAAAA#EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE	AS:i:0	XS:i:0
SRR12091997.46	77	*	0	0	*	*	0	0	CTGTTGGTGGGAATGTTTTGTATGCGTCAATATGCTTATTCAGCAAAATGACTTGATCTTTGAAATTTGGATCTTT	AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE	AS:i:0	XS:i:0
SRR12091997.48	77	*	0	0	*	*	0	0	GTCTGCCGAAAGCTTGTGTTACATTGTATGCTTTAGTGGCAGTACGTTTTTGCCGAGGCTTCTTAGAAGCCTCAG	AAAAAEEEEEEEEEEAEEAEEEEEEEAEEEEEEEEEEEEE/EEEEEAEEEEE<EEEEEEEEEAAEEEEAAEEEEE	AS:i:0	XS:i:0
sam

In [56]:
!samtools view -@ 24 -f 133 notebook_test_data/include_and_exclude/reads-to-recipient.bam | head -n 5

SRR12091997.6	141	*	0	0	*	*	0	0	GCTTTATCACTACCAAGAGTGTGTTAGAGGTACAACAGTACTTTTAAAAGAACCTTGCTCTTCTGGAACATACGAG	AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE	AS:i:0	XS:i:0
SRR12091997.33	141	*	0	0	*	*	0	0	GGAAATTTTGGGGACCAGGAACTAATCAGACAAGGAACTGATTACAAACATTGGCCGCAAATTGCACAATTTGCC	AAA/AEEEEEEEEEEEEEEEEEE/EEEEEEEEEEEEEE/EAEEEEEE/EEEAEEEEEAEAEEEAA/EEAA6AE/E	AS:i:0	XS:i:0
SRR12091997.34	141	*	0	0	*	*	0	0	ACACAGACCATTCCAGTAGCAGTGACAATATTGCTTTGCTTGTACAGTAAGTGACAACAGATGTTTCATCTCGTTG	AAAA6E6EEAEEAEAEEAEEAEEEEEEEEAE/EAEEEEEEEEEEEAEE6EEEEEEEEEEEAEEEEEAAEAA<AEEE	AS:i:0	XS:i:0
SRR12091997.46	141	*	0	0	*	*	0	0	GGAAATTTTGGGGACCAGGAACTAATCAGACAAGGAACTGATTACAAACATTGGCCGCAAATTGCACAATTTGCC	AAAAAEEEEEEEEEEAEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEE<	AS:i:0	XS:i:0
SRR12091997.48	141	*	0	0	*	*	0	0	AAGAAATTCAACTCCAGGCAGCAGTAGGGGAACTTCTCCTGCTAGAATGGCTGGCAATGGCGGTGATGCTGCTCTT	A/AAAAEEAE/EEEEEEEEE6/EEAEAAAEA/EEEEEAEEEE/AEAEEEEEEEEE/E<<AEEE/EEEEA6EEEEEE	AS:i:0	XS:i:0


In [57]:
sam_flag_to_bin(73)

73  : 000001001001
4022: 111110110110


In [58]:
sam_flag_to_bin(77)

77  : 000001001101
4018: 111110110010


In [59]:
sam_flag_decoder(77)

000001001101  1     read paired - template having multiple segments in sequencing
000001001101  0   
000001001101  4     read unmapped - segment unmapped
000001001101  8     mate unmapped - next segment in the template unmapped
000001001101  0   
000001001101  0   
000001001101  64    first in pair - the first segment in the template
000001001101  0   
000001001101  0   
000001001101  0   
000001001101  0   
000001001101  0   


'read paired - template having multiple segments in sequencing + read unmapped - segment unmapped + mate unmapped - next segment in the template unmapped + first in pair - the first segment in the template'

In [60]:
sam_flag_decoder(141)

000010001101  1     read paired - template having multiple segments in sequencing
000010001101  0   
000010001101  4     read unmapped - segment unmapped
000010001101  8     mate unmapped - next segment in the template unmapped
000010001101  0   
000010001101  0   
000010001101  0   
000010001101  128   second in pair - the last segment in the template
000010001101  0   
000010001101  0   
000010001101  0   
000010001101  0   


'read paired - template having multiple segments in sequencing + read unmapped - segment unmapped + mate unmapped - next segment in the template unmapped + second in pair - the last segment in the template'

In [63]:
!samtools view -@ 24 -f 73 -F 4022 notebook_test_data/include_and_exclude/reads-to-recipient.bam | head -n 5

SRR12091997.2260	73	chr2	32916258	6	76M	=	32916258	0	AGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG	AAA/AEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE<EEEEEEE/EEEEE	NM:i:1	MD:Z:0G75	AS:i:75	XS:i:71
SRR12091997.20261	73	chr16	8892814	60	76M	=	8892814	0	CCAGCAAAAAGGCAATATACAGGAAGAGTTGGTGTACACTGCTCCCACAGAAACAACCCAAAAAATACCGGAAAAG	AA/AAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEAEEEEEEEE/EEEE6EEE6EEEEEEEE6EEEEEEAEEEE	NM:i:0	MD:Z:76	AS:i:76	XS:i:0
SRR12091997.27348	73	chr5	172771120	60	46M30S	=	172771120	0	GGCTCCTCGGCTTCTTCGCGGTTCCCCCGACTGCCCCTCCGACCCGGTGATTTCTTTAGGCAGGTCCTTGATGTCA	AAAAAEEEEEEEEEEEEEEEEEEEEEEEE/EEEEEEEEEEE/EEEEE/E/EEEEEEE6EEEEEEEAAAEEEEAE/A	NM:i:0	MD:Z:46	AS:i:46	XS:i:0
SRR12091997.79038	73	chr20	17621737	60	40M35S	=	17621737	0	CTTTCTCTTTGAGATCCTGCAGCCACTCGGTGTAATTCTGTTGTGCCAAGACAGAGAGTTCTGGGAGCAGAGCCA	A/AAAE/EEEA//EE/EEAEAEEEE/E/A/EAE6EEAAEEEEEEEAAEEEE/EEE/AEEEA//EEEEEEEEAA<E	NM:i:0	MD:Z:40	AS:i:40	XS:i:0	SA:Z:chr20,17621855,+,37S38M,60,0;
SR

In [64]:
!samtools view -@ 24 -f 133 -F 3962 notebook_test_data/include_and_exclude/reads-to-recipient.bam | head -n 5

SRR12091997.2260	133	chr2	32916258	0	*	=	32916258	0	GCTCTTGCTTTGCTGCTGCTTGACAGATTGAACCAGCTTGAGAGCAAAATGTCTGGTAAAGGCCAACAACAACAAG	AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE	MC:Z:76M	AS:i:0	XS:i:0
SRR12091997.20261	133	chr16	8892814	0	*	=	8892814	0	TGGTGACGGTAAAATGAAAGATCTCAGTCCAAGATGGTATTTCTACTACCTAGGAACTGGGCCAGAAGCTGGACTT	AAAAA6EEEEEEEEEEEEEEAAEEAEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEAEEE	MC:Z:76M	AS:i:0	XS:i:0
SRR12091997.27348	133	chr5	172771120	0	*	=	172771120	0	AGTAACTTTAGCTTGTTTTGTGCTTGCTGCTGTTTACAGAATAAATTGGATCACCGGTGGAATTGCTATCGCAATG	AAAAAEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAAEEE	MC:Z:46M30S	AS:i:0	XS:i:0
SRR12091997.79038	133	chr20	17621737	0	*	=	17621737	0	GGAGAAGCTGCACTCCCTGACCCAGGCCAAGGAGGAATCGGAGAAGCAGCTCT	//A6AEEA/AEEE/A/A</EEAEE/EEAAEE/E6/EA/A</EEEE6EEA6//A	MC:Z:40M35S	AS:i:0	XS:i:0
SRR12091997.89182	133	chr10	100229427	0	*	=	100229427	0	CTGCTGTTAATAGTGTCCCTTGGGATACTATAGCTAATTATGCTAAGCCTTTTCTTAACAAAG

In [65]:
!samtools view -@ 24 notebook_test_data/include_and_exclude/reads-to-recipient.bam | grep 'SRR12091997.2260\s' | head -n 5

SRR12091997.2260	73	chr2	32916258	6	76M	=	32916258	0	AGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG	AAA/AEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE<EEEEEEE/EEEEE	NM:i:1	MD:Z:0G75	AS:i:75	XS:i:71
SRR12091997.2260	329	chr2	32916486	0	76M	=	32916486	0	*	*	NM:i:1	MD:Z:71A4	AS:i:71
SRR12091997.2260	329	chr2	32916351	0	76M	=	32916351	0	*	*	NM:i:1	MD:Z:50A25	AS:i:71
SRR12091997.2260	329	chr2	32916441	0	76M	=	32916441	0	*	*	NM:i:2	MD:Z:32A12A30	AS:i:66
SRR12091997.2260	133	chr2	32916258	0	*	=	32916258	0	GCTCTTGCTTTGCTGCTGCTTGACAGATTGAACCAGCTTGAGAGCAAAATGTCTGGTAAAGGCCAACAACAACAAG	AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE	MC:Z:76M	AS:i:0	XS:i:0


In [71]:
!samtools view -@ 24 notebook_test_data/include_and_exclude/reads-to-recipient_MU_R1.bam | grep 'SRR12091997.2260\s' | head -n 5

SRR12091997.2260	73	chr2	32916258	6	76M	=	32916258	0	AGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG	AAA/AEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE<EEEEEEE/EEEEE	NM:i:1	MD:Z:0G75	AS:i:75	XS:i:71


In [75]:
!samtools view -@ 24 notebook_test_data/include_and_exclude/reads-to-recipient_MU_R2.bam | grep 'SRR12091997.2260\s' | head -n 5

SRR12091997.2260	133	chr2	32916258	0	*	=	32916258	0	GCTCTTGCTTTGCTGCTGCTTGACAGATTGAACCAGCTTGAGAGCAAAATGTCTGGTAAAGGCCAACAACAACAAG	AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE	MC:Z:76M	AS:i:0	XS:i:0


In [76]:
!samtools view -@ 24 notebook_test_data/include_and_exclude/reads-to-recipient_MU.bam | grep 'SRR12091997.2260\s' | head -n 5

SRR12091997.2260	73	chr2	32916258	6	76M	=	32916258	0	AGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG	AAA/AEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE<EEEEEEE/EEEEE	NM:i:1	MD:Z:0G75	AS:i:75	XS:i:71
SRR12091997.2260	133	chr2	32916258	0	*	=	32916258	0	GCTCTTGCTTTGCTGCTGCTTGACAGATTGAACCAGCTTGAGAGCAAAATGTCTGGTAAAGGCCAACAACAACAAG	AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAE	MC:Z:76M	AS:i:0	XS:i:0


In [81]:
!samtools view -@ 24 notebook_test_data/include_and_exclude/reads-to-recipient.bam | grep SRR12091997.16771731

SRR12091997.16771731	81	chr12	57527847	60	18S57M	chr2	32916484	0	CTGGGTGCCAGCCTGCTGGGTGACCTGTCTTCACTGACCAGCAGCCCTGGAGCCCTCCCCAGCCTGTTGCAGCCG	AEEAEAEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEAEEEEEEEEEEEEEEEEAEEEAAAAA	NM:i:1	MD:Z:56T0	MC:Z:30S46M	AS:i:56	XS:i:19
SRR12091997.16771731	161	chr2	32916484	1	30S46M	chr12	57527847	0	TTTAAATTCTGCGCTGCTGGCTGCCACCCTGGAGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG	AAAAAEEEEEEEEEAEEEEEAEEEEAEAEEAE/EEAE6/EA/EAEEAEEEEE/E/EEEEEAE/E/AEEEEEEEEEE	NM:i:0	MD:Z:46	MC:Z:18S57M	AS:i:46	XS:i:45	SA:Z:chr12,57527170,+,33M43S,1,0;
SRR12091997.16771731	417	chr2	32916350	0	31H45M	chr12	57527847	0	*	*	NM:i:0	MD:Z:45	MC:Z:18H57M	AS:i:45
SRR12091997.16771731	417	chr2	32916399	0	30H46M	chr12	57527847	0	*	*	NM:i:1	MD:Z:42A3	MC:Z:18H57M	AS:i:42
SRR12091997.16771731	2209	chr12	57527170	1	33M43H	=	57527847	734	TTTAAATTCTGCGCTGCTGGCTGCCACCCTGGA	AAAAAEEEEEEEEEAEEEEEAEEEEAEAEEAE/	NM:i:0	MD:Z:33	MC:Z:18H57M	AS:i:33	XS:i:19	SA:Z:chr2,32916484,+,30S46M,1,0;
